#Feature extraction from signature images

##Unzip the file

In [ ]:
# Step 1: Install unrar if not already installed
!apt-get install -y unrar

# Step 2: Create a directory to extract the files
!mkdir -p /content/extracted_files

# Step 3: Extract the RAR file to the specified directory
!unrar x /content/Reference.rar /content/extracted_files/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/Reference.rar

Creating    /content/extracted_files/Reference                        OK
Extracting  /content/extracted_files/Reference/R001.png                    3%  OK 
Extracting  /content/extracted_files/Reference/R002.png                    6%  7%  OK 
Extracting  /content/extracted_files/Reference/R003.png                   10% 11%  OK 
Extracting  /content/extracted_files/Reference/R004.png                   14% 15%  OK 
Extracting  /content/extracted_files/Reference/R005.png                   18%  OK 
Extracting  /content/extracted_files/Reference/R006.png                   22% 23%  OK 
Extracting  /cont

##Convertng image into black and white

In [ ]:
import os
from PIL import Image

# Create a directory to save the black-and-white images
os.makedirs('/content/black_and_white_images', exist_ok=True)

# Path to the folder with extracted images
source_folder = '/content/extracted_files/Reference'
destination_folder = '/content/black_and_white_images'

# Convert each image to black and white and save
for filename in os.listdir(source_folder):
    if filename.endswith(".png"):
        # Open the image file
        image_path = os.path.join(source_folder, filename)
        image_file = Image.open(image_path)

        # Convert the image to black and white
        bw_image = image_file.convert('1')

        # Save the black-and-white image
        bw_image.save(os.path.join(destination_folder, filename))

print("Conversion to black and white is completed. Images saved in /content/black_and_white_images/")


Conversion to black and white is completed. Images saved in /content/black_and_white_images/


##Drawing Bounding Box around the images

In [ ]:
from PIL import Image, ImageDraw
import os

# Folder paths
source_folder = '/content/black_and_white_images'
bounding_box_folder = '/content/bounding_box_images'

# Create folder to save images with bounding boxes
os.makedirs(bounding_box_folder, exist_ok=True)

# Function to find the bounding box
def find_bounding_box(image):
    width, height = image.size
    left, right, top, bottom = width, 0, height, 0

    # Iterate through each pixel
    for x in range(width):
        for y in range(height):
            color = image.getpixel((x, y))
            if color == 0:  # Pixel is black
                if x > right:
                    right = x
                if x < left:
                    left = x
                if y > bottom:
                    bottom = y
                if y < top:
                    top = y
    return (left, top, right, bottom)

# Iterate over each black-and-white image and draw a bounding box
for filename in os.listdir(source_folder):
    if filename.endswith(".png"):
        image_path = os.path.join(source_folder, filename)
        image = Image.open(image_path)

        # Find the bounding box
        bbox = find_bounding_box(image)

        if bbox:
            # Draw the bounding box on the original image
            draw = ImageDraw.Draw(image)
            draw.rectangle(bbox, outline="black")

            # Save the image with the bounding box
            save_path = os.path.join(bounding_box_folder, filename)
            image.save(save_path)

print("Bounding boxes have been drawn and saved in /content/bounding_box_images/")

Bounding boxes have been drawn and saved in /content/bounding_box_images/


##Finding centroid of the images

In [ ]:
from PIL import Image, ImageDraw
import os

# Folder paths
bounding_box_folder = '/content/bounding_box_images'
centroid_folder = '/content/centroid_images'

# Create folder to save images with centroids
os.makedirs(centroid_folder, exist_ok=True)

# Function to calculate the centroid of black pixels
def find_centroid(image):
    width, height = image.size
    cx, cy, n = 0, 0, 0

    # Iterate through each pixel
    for x in range(width):
        for y in range(height):
            color = image.getpixel((x, y))
            if color == 0:  # Pixel is black
                cx += x
                cy += y
                n += 1

    # Calculate the centroid if there are black pixels
    if n > 0:
        cx /= n
        cy /= n
        return (cx, cy)
    else:
        return None  # Return None if there are no black pixels

# Iterate over each black-and-white image, draw the centroid if found
for filename in os.listdir(bounding_box_folder):
    if filename.endswith(".png"):
        image_path = os.path.join(bounding_box_folder, filename)
        image = Image.open(image_path)

        # Find the centroid
        centroid = find_centroid(image)

        if centroid:
            # Draw the centroid on the image
            draw = ImageDraw.Draw(image)
            cx, cy = centroid
            # Mark the centroid as a small red circle
            draw.ellipse((cx-30, cy-30, cx+30, cy+30), fill="black", outline="black")

        # Save the image with the centroid marked
        save_path = os.path.join(centroid_folder, filename)
        image.save(save_path)

print("Centroids have been drawn and saved in /content/centroid_images/")


Centroids have been drawn and saved in /content/centroid_images/


##Segmenting Images into four cells

In [ ]:
from PIL import Image, ImageDraw
import os

# Folder paths
source_folder = '/content/centroid_images'
seg_folder = '/content/segmented_image'

# Create folder to save images with bounding boxes
os.makedirs(seg_folder, exist_ok=True)

# Iterate over each black-and-white image and draw a bounding box
for filename in os.listdir(source_folder):
    if filename.endswith(".png"):
        image_path = os.path.join(source_folder, filename)
        image = Image.open(image_path)

        # Find the bounding box
        bbox = find_bounding_box(image)
        centroid = find_centroid(image)
        cx, cy = centroid
        left, top, right, bottom = bbox

        # Draw the bounding box on the original image
        draw = ImageDraw.Draw(image)
        # Top-left to centroid
        draw.rectangle((left, top, cx, cy), outline="black")
        # Top-right to centroid
        draw.rectangle((cx, top, right, cy), outline="black")
        # Bottom-left to centroid
        draw.rectangle((left, cy, cx, bottom), outline="black")
        # Bottom-right to centroid
        draw.rectangle((cx, cy, right, bottom), outline="black")

        # Save the image with the bounding box
        save_path = os.path.join(seg_folder, filename)
        image.save(save_path)

print("Segmented boxes have been drawn and saved in /content/segmented_image/")

Segmented boxes have been drawn and saved in /content/segmented_image/


##Finding white to black tranistion in each of those cells

In [ ]:
from PIL import Image, ImageDraw
import os


def find_Transitions(image, width, height):
    n = 0
    # Iterate through each pixel
    prev = image.getpixel((0, 0))
    # Convert width and height to integers before using them in range
    for x in range(int(width)):
        for y in range(int(height)):
            color = image.getpixel((x, y))
            if color == 255 and prev == 0:  # Pixel is black
                n += 1
            prev = color
    return n




# Folder paths
source_folder = '/content/segmented_image'


# # Create folder to save images with bounding boxes
# os.makedirs(seg_folder, exist_ok=True)

# Iterate over each black-and-white image and draw a bounding box
for filename in os.listdir(source_folder):
    if filename.endswith(".png"):
        image_path = os.path.join(source_folder, filename)
        image = Image.open(image_path)

        # Find the bounding box
        bbox = find_bounding_box(image)
        centroid = find_centroid(image)
        cx, cy = centroid
        left, top, right, bottom = bbox

        #top-left
        TL = find_Transitions(image, cx - left, cy - top)
        #top right
        TR = find_Transitions(image, right - cx, cy - top)
        #bottom left
        BL = find_Transitions(image, cx - left, bottom - cy)
        #bottom right
        BR = find_Transitions(image, right - cx, bottom - cy)

        transitions = (TL, TR, BL, BR)
        print(transitions)

(20055, 21705, 27100, 30041)
(19507, 21662, 29925, 33656)
(18141, 19394, 25243, 27091)
(18946, 21673, 31273, 35464)
(21842, 24268, 33614, 38338)
(18598, 19803, 28632, 30972)
(18660, 20654, 28175, 31106)
(19467, 19480, 30204, 30221)
(17712, 19114, 27724, 30428)
(17798, 20287, 27190, 31263)
(18344, 20615, 21612, 24260)
(22923, 24013, 30035, 31601)
(22026, 23361, 30838, 33504)
(17024, 17741, 26755, 27862)
(20044, 20896, 26824, 28327)
(19919, 24232, 32960, 39265)
(20889, 22147, 28788, 30903)
(16530, 17349, 26850, 28291)
(20636, 21177, 23969, 24761)
(23089, 24292, 25196, 26689)
(18385, 19320, 26431, 27940)
(18224, 19677, 27972, 30918)
(21745, 23177, 30908, 33350)
(15692, 15862, 23122, 23444)
(17497, 19079, 27205, 29596)


##Function to calculate the centroid using coordinate of each cell

In [ ]:
# Function to calculate the centroid of black pixels
def findCentroid(image, left, right, top, bottom):
    cx, cy, n = 0, 0, 0

    # Iterate through each pixel
    for x in range(int(left), int(right)):
        for y in range(int(top), int(bottom)):
            color = image.getpixel((x, y))
            if color == 0:  # Pixel is black
                cx += x
                cy += y
                n += 1

    # Calculate the centroid if there are black pixels
    if n > 0:
        cx /= n
        cy /= n
        return (cx, cy)
    else:
         return ((left + right) / 2, (top + bottom) / 2)

##Function to find the transitions using coordinates of each cell

In [ ]:
def findTransitions(image, left, right, top, bottom):
    n = 0

    # Iterate through each pixel
    prev = image.getpixel((int(left), int(top)))
    # Convert width and height to integers before using them in range
    for x in range(int(left), int(right)):
        for y in range(int(top), int(bottom)):
            color = image.getpixel((x, y))
            if color == 255 and prev == 0:  # Pixel is black
                n += 1
            prev = color

    return n

##Function to calculate aspect ratio of each cell

In [ ]:
def find_ratio(left, right, top, bottom):
  width = right - left
  height = bottom - top
  return (width/height)

##Function to draw outline on each cell

In [ ]:
def draw_cells(image, filename = "coordinates_file.txt"):
    with open(filename, 'r') as f:
        lines = f.readlines()

    draw = ImageDraw.Draw(image)
    i = 0
    while i < len(lines):
        if lines[i].startswith("Left"):
            # Extract coordinates
            parts = lines[i].strip().split(', ')
            coordinates = {}
            for part in parts:
                key, value = part.split(': ')
                coordinates[key] = int(float(value))

            left = coordinates['Left']
            right = coordinates['Right']
            top = coordinates['Top']
            bottom = coordinates['Bottom']

            # Draw the rectangle for this cell
            draw.rectangle((left, top, right, bottom), outline="black")
        i += 1

##Splitting Image into 64 segments and finding transition, coordinates, centroid and aspect ratio of each of the 64 cells
All this is saved in drawn_cells folder.

In [ ]:
import os
from PIL import Image, ImageDraw

# Function to split the image into 64 cells and save data in text files
def split_64(image, left, right, top, bottom, depth=0, f1="coords.txt", f2="centroids.txt", f3="ratios.txt", f4="trans.txt"):
    cx, cy = findCentroid(image, left, right, top, bottom)

    if depth < 3:  # Keep splitting until 64 cells
        split_64(image, left, cx, top, cy, depth + 1, f1, f2, f3, f4)
        split_64(image, cx, right, top, cy, depth + 1, f1, f2, f3, f4)
        split_64(image, left, cx, cy, bottom, depth + 1, f1, f2, f3, f4)
        split_64(image, cx, right, cy, bottom, depth + 1, f1, f2, f3, f4)
    else:
        t = findTransitions(image, left, right, top, bottom)
        r = find_ratio(left, right, top, bottom)

        # Save the data in different text files
        with open(f1, 'a') as f_coords:
            f_coords.write(f"Left: {left}, Right: {right}, Top: {top}, Bottom: {bottom}\n")

        with open(f4, 'a') as f_transitions:
            f_transitions.write(f"Transitions: {t}\n")

        with open(f3, 'a') as f_ratios:
            f_ratios.write(f"Aspect Ratio: {r}\n")

        with open(f2, 'a') as f_centroids:
            f_centroids.write(f"Centroid: ({cx}, {cy})\n")

# Folder paths
source_folder = '/content/bounding_box_images'
output_folder = '/content/drawn_cells'

os.makedirs(output_folder, exist_ok=True)

# Process each image
for filename in os.listdir(source_folder):
    if filename.endswith(".png"):
        image_path = os.path.join(source_folder, filename)
        image = Image.open(image_path)

        # Create a unique folder for each image within output_folder
        image_folder = os.path.join(output_folder, filename.split('.')[0])
        os.makedirs(image_folder, exist_ok=True)

        # Define paths for the data files within the image folder
        cell_coordinates = os.path.join(image_folder, "cell_coordinates.txt")
        cell_centroid = os.path.join(image_folder, "cell_centroid.txt")
        cell_aspectratio = os.path.join(image_folder, "cell_aspectratio.txt")
        cell_transition = os.path.join(image_folder, "cell_transitions.txt")

        # Split the image into 64 cells and save the data
        split_64(image, 0, image.width, 0, image.height, depth=0,
                 f1=cell_coordinates, f2=cell_centroid, f3=cell_aspectratio, f4=cell_transition)

        # Load the data from the file and draw the cells on the image
        draw_cells(image, filename=cell_coordinates)

        # Save the image with drawn cells within the image folder
        image_save_path = os.path.join(image_folder, filename)
        image.save(image_save_path)

print("Splitting, saving cell data, and drawing cells completed.")


Splitting, saving cell data, and drawing cells completed.


# Identification of Stable Cells

In [ ]:
import os
from PIL import Image
import numpy as np

# Function to store transitions for all signatures
def gather_transitions_for_signatures(source_folder, num_signatures):
    # Store transitions data for each cell across all signatures
    all_transitions = []

    for i, filename in enumerate(sorted(os.listdir(source_folder))):
        if i >= num_signatures:
            break

        if filename.endswith(".png"):
            image_path = os.path.join(source_folder, filename)
            image = Image.open(image_path)

            # Collect transition counts for this image
            transitions = []
            collect_transitions(image, 0, image.width, 0, image.height, transitions)
            all_transitions.append(transitions)

    return all_transitions

# Recursive function to collect transitions for 64 cells
def collect_transitions(image, left, right, top, bottom, transitions, depth=0):
    if depth < 3:  # Keep splitting until 64 cells
        cx, cy = findCentroid(image, left, right, top, bottom)
        collect_transitions(image, left, cx, top, cy, transitions, depth + 1)
        collect_transitions(image, cx, right, top, cy, transitions, depth + 1)
        collect_transitions(image, left, cx, cy, bottom, transitions, depth + 1)
        collect_transitions(image, cx, right, cy, bottom, transitions, depth + 1)
    else:
        t = findTransitions(image, left, right, top, bottom)
        transitions.append(t)

# Function to identify stable cells based on variance across all signatures
def identify_stable_cells(all_transitions, threshold=5):
    num_cells = 64
    stable_cells = [True] * num_cells

    # Compare transitions for each cell across all signatures
    for cell_index in range(num_cells):
        # Gather transitions for this cell across all signatures
        cell_transitions = [signature[cell_index] for signature in all_transitions]

        # Check the range of transition values or use standard deviation
        if max(cell_transitions) - min(cell_transitions) > threshold:
            stable_cells[cell_index] = False

    return stable_cells

# Main function to compare and mark stable cells across signatures
def compare_and_mark_stable_cells(source_folder, num_signatures=25):
    # Gather transitions for all signatures
    all_transitions = gather_transitions_for_signatures(source_folder, num_signatures)

    # Identify stable cells
    stable_cells = identify_stable_cells(all_transitions, threshold=5)

    # Output results
    with open('stable_cells.txt', 'w') as f:
        for i, is_stable in enumerate(stable_cells):
            f.write(f"Cell {i+1}: {'Stable' if is_stable else 'Unstable'}\n")

    print("Stable cell analysis completed.")

# Example Usage
source_folder = '/content/bounding_box_images'
compare_and_mark_stable_cells(source_folder, num_signatures=25)

Stable cell analysis completed.


##Calculating the Slant angle of the signature using vertical projection profile

In [ ]:
import numpy as np
from PIL import Image, ImageOps
import math

# Function to perform vertical shear transformation
def shear_image(image, angle):
    angle_rad = math.radians(angle)
    tan_angle = math.tan(angle_rad)
    width, height = image.size

    # Calculate new image size after shear
    new_width = int(width + abs(tan_angle) * height)
    new_image = ImageOps.expand(image, (new_width - width, 0, 0, 0), fill="white")

    # Apply shear transformation
    return new_image.transform(new_image.size, Image.AFFINE, (1, tan_angle, -tan_angle * height / 2, 0, 1, 0), Image.BICUBIC)

# Function to calculate vertical projection profile
def vertical_projection_profile(image):
    # Convert to numpy array and calculate column-wise sum
    image_array = np.array(image)
    projection_profile = np.sum(image_array == 0, axis=0)  # Summing black pixels column-wise
    return projection_profile

# Function to estimate slant angle using vertical projection profiles
def estimate_slant_angle(image, shear_range=(-10, 10), shear_increment=1):
    max_projection = 0
    optimal_shear_angle = 0

    for angle in range(shear_range[0], shear_range[1] + 1, shear_increment):
        # Shear the image and compute vertical projection profile
        sheared_image = shear_image(image, angle)
        current_projection = np.sum(vertical_projection_profile(sheared_image))

        # Update maximum projection and optimal shear angle
        if current_projection > max_projection:
            max_projection = current_projection
            optimal_shear_angle = angle

    return optimal_shear_angle

##Calculating the skew angle using horizontal projection profile

In [ ]:
import numpy as np
from PIL import Image
import math

# Function to calculate horizontal projection profile
def horizontal_projection_profile(image):
    # Convert to numpy array and calculate row-wise sum
    image_array = np.array(image)
    projection_profile = np.sum(image_array == 0, axis=1)  # Summing black pixels row-wise
    return projection_profile

# Function to estimate skew angle
def estimate_skew(image, initial_angle=0, rotation_increment=5, max_iterations=10):
    # Step 2: Initial horizontal projection profile
    max_projection = np.sum(horizontal_projection_profile(image))
    optimal_angle = initial_angle

    for i in range(max_iterations):
        # Rotate by the specified increment
        rotated_image = image.rotate(optimal_angle + rotation_increment, expand=True)

        # Step 5: Calculate projection profile and sum of rotated image
        current_projection = np.sum(horizontal_projection_profile(rotated_image))

        # Step 6: If the current projection is greater than max, update the angle and max_projection
        if current_projection > max_projection:
            max_projection = current_projection
            optimal_angle += rotation_increment
        else:
            # Step 7: If no improvement, break out
            break

    return optimal_angle

##Dumping angles to file

In [ ]:
# Folder paths
source_folder = '/content/bounding_box_images'
output_folder = '/content/corrected_slant_cells'
os.makedirs(output_folder, exist_ok=True)

# Process each image
for filename in os.listdir(source_folder):
    if filename.endswith(".png"):
        image_path = os.path.join(source_folder, filename)
        image = Image.open(image_path)


        # Estimate skew and rotate image accordingly
        skew_angle = estimate_skew(image)
        slant_angle = estimate_slant_angle(image)

        fname = filename.split('.')[0]
        # Define the output file path
        output_file_path = os.path.join(output_folder, f"{fname}_angles.txt")

    # Write skew and slant angles to file
    with open(output_file_path, 'w') as f:
        f.write(f"Skew Angle: {skew_angle} degrees\n")
        f.write(f"Slant Angle: {slant_angle} degrees\n")





print("Skew angle correction completed.")

Skew angle correction completed.


## Lab Tasks Summary

### Feature Extraction from Signature Images
1. **Unzipping Files**:
   - Installed `unrar` and extracted images from a RAR file to a specified directory.

2. **Converting Images to Black and White**:
   - Converted each image to black and white and saved them in a new directory.

### Drawing Bounding Boxes
3. **Bounding Box Calculation**:
   - Calculated the bounding box for each black-and-white image and drew the bounding box on the images.

### Finding Centroids
4. **Centroid Calculation**:
   - Calculated the centroid of black pixels in each image and marked it with a small circle.

### Segmenting Images into Four Cells
5. **Image Segmentation**:
   - Divided each image into four segments using the centroid and bounding box coordinates.

### Finding White to Black Transitions
6. **Transition Calculation**:
   - Counted the transitions from white to black pixels in each of the four segments.

### Calculating Centroids and Transitions for Each Cell
7. **Cell-wise Calculations**:
   - Calculated centroids and transitions for each of the 64 cells obtained by further dividing the four segments.

### Calculating Aspect Ratios
8. **Aspect Ratio Calculation**:
   - Calculated the aspect ratio for each of the 64 cells.

### Drawing Outlines on Cells
9. **Cell Outlining**:
   - Drew outlines around each of the 64 cells and saved the images.

### Calculating Slant and Skew Angles
10. **Slant Angle Calculation**:
    - Estimated the slant angle of the signature using vertical projection profiles.

11. **Skew Angle Calculation**:
    - Estimated the skew angle using horizontal projection profiles and corrected the images accordingly.

### Saving Results
12. **Data Dumping**:
    - Saved the calculated angles (skew and slant) to text files for each image.

---

These tasks collectively aimed at processing and analyzing signature images for feature extraction, segmentation, and geometric corrections.
